In [21]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

import dataset
assert tf.config.list_physical_devices('GPU')

In [112]:
BATCH_SIZE = 32
MAXLEN = 64

def mask_random(original):
    noised = np.copy(original)
    nrows = noised.shape[0]
    for row in range(nrows):
        p = min(0.99, row / (nrows * (1/32)))
        for i in range(noised.shape[1]):
            if np.random.rand() < p:
                noised[row, i] = 1
#         indices = np.random.choice(np.arange(noised.size), replace=False, size=int(noised.size / MAXLEN), p=np.linspace(0.0, 1.0, noised.size))
#         indices = np.unravel_index(indices, noised.shape)
#         noised[indices] = 0|
    return noised


def build_model():
    EMBED_DIM = 512
    UNITS = 256

    LETTERS_SIZE = len(dataset.letters_table)
    NIQQUD_SIZE = len(dataset.niqqud_table)
    DAGESH_SIZE = len(dataset.dagesh_table)
    SIN_SIZE = len(dataset.sin_table)

    input_niqqud = tf.keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    niqqud_layer = layers.Embedding(NIQQUD_SIZE, NIQQUD_SIZE, mask_zero=True)(input_niqqud)
    
    layer = input_text = tf.keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    
    layer = layers.Embedding(LETTERS_SIZE, EMBED_DIM, mask_zero=True)(layer)
    layer = layers.concatenate([layer, niqqud_layer])
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(layer)
    layer = layers.add([layer,
                         layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer))
    ]
    model = tf.keras.Model(inputs=[input_text, input_niqqud], outputs=outputs)

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    tf.keras.utils.plot_model(model, to_file='model.png')
    model.summary()
    return model
    
model = build_model()
model.save_weights('./checkpoints/uninit')

def fit(data, learning_rates):
    x  = [data.normalized_texts, mask_random(data.niqqud_texts)]
    vx = [data.normalized_validation, np.ones(data.niqqud_validation.shape)]
    y  = {'N': data.niqqud_texts,      'D': data.dagesh_texts,      'S': data.sin_texts,      'C': data.normalized_texts     }
    vy = {'N': data.niqqud_validation, 'D': data.dagesh_validation, 'S': data.sin_validation, 'C': data.normalized_validation}
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=len(learning_rates),
          callbacks=[
              tf.keras.callbacks.LearningRateScheduler(lambda epoch, lr: learning_rates[epoch], verbose=0),
          ]
    )

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
Model: "model_31"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_80 (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_79 (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding_72 (Embedding)        (None, 64, 512)      22528       input_80[0][0]                   
__________________________________________________________________________________________________
embedding_71 (Embedding)        (None, 64, 16)       256         input_79[0][0]                   
_____

In [46]:
def load_data(source, maxlen=MAXLEN):
    filenames = [os.path.join('texts', f) for f in source]
    return dataset.load_file(filenames, BATCH_SIZE, 0.1, maxlen=maxlen, shuffle=True)

In [4]:
data_rabanit = load_data(['birkat_hamazon.txt', 'kuzari.txt', 'hakdama_leorot.txt', 'hartzaat_harav.txt', 'orhot_hayim.txt', 'rambam_mamre.txt', 'short_table.txt', 'tomer_dvora.txt'])

In [5]:
data_pre_modern = load_data(['elef_layla.txt', 'bialik', 'shaul_tchernichovsky', 'breslev.txt', 'itzhak_berkman', 'zevi_scharfstein', 'pesah_kaplan', 'abraham_regelson',
                             'elisha_porat', 'uriel_ofek', 'yisrael_dushman', 'zvi_zviri', 'atar_hashabat.txt', 'ali_baba.txt'])

In [47]:
data_modern = load_data(['forums', 'newspapers', 'wiki', 'blogs', 'adamtsair.txt', 'katarsis.txt'])  # , 'imagination.txt', 'sipurim.txt', 'ricky.txt'

In [ ]:
model.load_weights('./checkpoints/uninit')
history = fit(data_rabanit, [3e-3, 3e-4])
model.save_weights('./checkpoints/rabanit')

Train on 76377 samples, validate on 8487 samples
Epoch 1/2
76377/76377 [==============================] - 111s 1ms/sample - loss: 0.2882 - N_loss: 0.2230 - D_loss: 0.0578 - S_loss: 0.0074 - N_accuracy: 0.9224 - D_accuracy: 0.9781 - S_accuracy: 0.9980 - val_loss: 0.1525 - val_N_loss: 0.1152 - val_D_loss: 0.0349 - val_S_loss: 0.0029 - val_N_accuracy: 0.9608 - val_D_accuracy: 0.9873 - val_S_accuracy: 0.9993
Epoch 2/2
72512/76377 [===========================>..] - ETA: 4s - loss: 0.1129 - N_loss: 0.0838 - D_loss: 0.0267 - S_loss: 0.0024 - N_accuracy: 0.9716 - D_accuracy: 0.9904 - S_accuracy: 0.9994

In [ ]:
model.load_weights('./checkpoints/rabanit')
history = fit(data_pre_modern, [3e-3, 3e-4])
model.save_weights('./checkpoints/pre_modern')

In [ ]:
model.load_weights('./checkpoints/pre_modern')
history = fit(data_modern, [3e-3, 3e-4])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [18]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, '.')

In [ ]:
def print_predictions(data, k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.normalized_validation[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud_validation[s], data.dagesh_validation[s], data.sin_validation[s]]
    actual = data.merge(batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = data.merge(batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        print()

print_predictions(load_data(['blogs'], maxlen=MAXLEN), 0)